In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 

In [28]:
# Checking if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [29]:
# Loading the data
train_data = pd.read_csv("datasetTV.csv")
test_data = pd.read_csv("datasetTest.csv")


In [30]:
# Preprocess Data
X_train = train_data.iloc[:, :-1].values  # Features
y_train = train_data.iloc[:, -1].values   # Labels
X_test = test_data.values                 # Test features (no labels provided)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train - 1, dtype=torch.long)  
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# Putting all the tensors to the device
X_train_tensor = X_train_tensor.to(device)  
y_train_tensor = y_train_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)


In [31]:
# Create a Model class that inherits from nn.Module
class Model(nn.Module):
    # Input layer with 8743 samples 
    # Each sample has 224 features and 1 label that takes the values 1, 2, 3, 4 and 5
    def __init__(self, in_features, h1, h2, num_classes):
        super(Model, self).__init__() # Instantiate the parent class
        self.fc1 = nn.Linear(in_features, h1)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, num_classes)
        

    def forward(self, x):
        x = F.relu(self.fc1(x))
        X = F.relu(self.fc2(x))
        X = self.out(x)
        return x

In [35]:
torch.manual_seed(42)
model = Model(in_features=224, h1=64, h2=64, num_classes=5).to(device)

In [38]:
# Preping for training
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [39]:
# Training the model
epochs = 100
losses = []
for i in range(epochs):
    optimizer.zero_grad()
    y_pred = model(X_train_tensor)
    loss = criteria(y_pred, y_train_tensor)
    losses.append(loss)
    loss.backward()
    optimizer.step()
    if i % 10 == 0:
        print(f'Epoch: {i} Loss: {loss.item()}')

Epoch: 0 Loss: 0.6825825572013855
Epoch: 10 Loss: 0.6758037805557251
Epoch: 20 Loss: 0.6700941324234009
Epoch: 30 Loss: 0.6650980710983276
Epoch: 40 Loss: 0.6606721878051758
Epoch: 50 Loss: 0.6566831469535828
Epoch: 60 Loss: 0.6530754566192627
Epoch: 70 Loss: 0.6498700380325317
Epoch: 80 Loss: 0.6470834612846375
Epoch: 90 Loss: 0.6444001197814941
